In [1]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np

In [2]:
def isNested(entry): 
    try: 
        sum(entry)
        return 0 
    except: 
        return 1

In [3]:
def readInJson(fileName):
    try: 
        df = pd.read_json(fileName)
    except: 
        handle = json.load(open(fileName))
        df = pd.DataFrame(handle['features'])
        
    return df

In [4]:
garbageCollectionDate = sorted([i for i in os.listdir() if i.endswith('json')])

In [5]:
lonlatSuburb = pd.read_csv('lonlatSuburb.csv')
vicPostcodes = pd.read_csv('vicPostcodes.csv')

lonlatSuburb['lat'] = lonlatSuburb.lat.apply(lambda x: round(x, 5))
lonlatSuburb['long'] = lonlatSuburb.long.apply(lambda x: round(x, 5))

vicPostcodes['lat'] = vicPostcodes.lat.apply(lambda x: round(x, 6))
vicPostcodes['long'] = vicPostcodes.long.apply(lambda x: round(x, 6))

In [6]:
garbageCollectionDate[21]

'greatDandenong-garbage-collection-zones.json'

In [16]:
df = readInJson(garbageCollectionDate[21])

df['geo'] = df.geometry.apply(lambda x: x['coordinates'][0][0])

df = df.explode('geo')

df['geo_nested'] = df.geo.apply(lambda x: isNested(x))
df['lat'] = df.geo.apply(lambda x: round(x[0], 5))
df['long'] = df.geo.apply(lambda x: round(x[1], 5))

df.drop(['geo','id', 'geometry', 'geometry_name',
         'geo_nested', 'type'], axis = 1, inplace = True)

In [17]:
df.iloc[0,0]

{'name': 'kml_1',
 'snippet': '',
 'garbage_day': 'TUESDAY',
 'week': '1',
 'area_colour': 'Red - Area 1',
 'binlid_colour': 'Red'}

In [18]:
def collectionFrequency(row, wasteType): 
    
    rub_colDay = row[f'{wasteType}_day']
    rub_colFreq = row['week']
    
    if rub_colFreq == '2' or rub_colFreq == 2: 
        rub_colFreq = 'Every Second '
    else: 
        rub_colFreq = 'Every '
        
    if rub_colDay in [None, 'None']:
        rub_colDay = ''
    elif rub_colFreq == [None, 'None']: 
        rub_colFreq = ''
        
    returningStr = rub_colFreq + rub_colDay.capitalize()
    
    if len(returningStr.split())>1:
         return returningStr
    else: 
        return 'Unavailable'

df['landfill_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'garbage'))

df.drop('properties', axis = 1, inplace = True)

In [19]:
dandenong = lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay']]\
.sort_values(['locality', 'landfill_colDay'])\
        .drop_duplicates(['locality','landfill_colDay'])\
        .sort_values(['locality'])\
.merge(vicPostcodes[['locality', 'postcode']])\
.sort_values(['postcode', 'landfill_colDay'])\
.drop_duplicates(subset = ['postcode', 'landfill_colDay'])\
.rename(columns = dict(zip(['landfill_colDay'], 
                           ['landfill'])))\
.drop(['long', 'lat', 'locality'], axis = 1)\
.melt(id_vars = 'postcode')\
.rename(columns = {'variable':'binInfo_bin_type', 'value':'collection_time'})\
.drop_duplicates(['postcode', 'binInfo_bin_type'])

In [21]:
dandenong.to_csv('../GBD_dandenong.csv', index = False)